In [1]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler matplotlib_venn goatools gseapy scperturb biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 41.0.7 which is incompatible.


In [2]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../')
from utils import *

# scperturb package
sys.path.insert(1, '../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../figures/')

/tmp/ipykernel_82/1400239515.py:17: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')

In [ ]:
path = DIR + 'GSE149383/supp/'
# format: ['GEO', 'days', 'origin', 'condition', 'bundle', 'subseries']

In [113]:
X=[
# GSE134836 (Erl PC9) is weird
# ['GSM3972651', 0, 'PC9', 'control', 'PC9D0vsD3Erl', 'GSE134836'],  #PC9D0
# ['GSM3972652', 3, 'PC9', 'Erl', 'PC9D0vsD3Erl', 'GSE134836'],  #PC9D3Erl
# GSE134838 (Vem M14)
['GSM3972655', 0, 'M14', 'control', 'M14D0vsD3Vem', 'GSE134838'], #M14Day0
['GSM3972656', 3, 'M14', 'Vem', 'M14D0vsD3Vem', 'GSE134838'], #M14Day3_Vem
# GSE134839 (Erl PC9)
['GSM3972657', 0, 'PC9', 'Erl', 'D0_D1_D2_D4_D9_D11', 'GSE134839'], #Day 0 PC9 erlotinib
['GSM3972658', 1, 'PC9', 'Erl', 'D0_D1_D2_D4_D9_D11', 'GSE134839'], #Day 1 PC9 erlotinib
['GSM3972659', 2, 'PC9', 'Erl', 'D0_D1_D2_D4_D9_D11', 'GSE134839'], #Day 2 PC9 erlotinib
['GSM3972660', 4, 'PC9', 'Erl', 'D0_D1_D2_D4_D9_D11', 'GSE134839'], #Day 4 PC9 erlotinib
['GSM3972661', 9, 'PC9', 'Erl', 'D0_D1_D2_D4_D9_D11', 'GSE134839'], #Day 9 PC9 erlotinib
['GSM3972662', 11, 'PC9', 'Erl', 'D0_D1_D2_D4_D9_D11', 'GSE134839'], #Day 11 PC9 erlotinib
# GSE134841 (Drug holiday run Erl PC9)
['GSM3972669', 0, 'PC9', 'Erl', 'Drug_holiday', 'GSE134841'], #PC9 Day 0
['GSM3972670', 2, 'PC9', 'Erl', 'Drug_holiday', 'GSE134841'], #PC9 Day 2
['GSM3972671', 11, 'PC9', 'Erl', 'Drug_holiday', 'GSE134841'], #PC9 Day 11
['GSM3972672', 19, 'PC9', 'Erl_DMSO', 'Drug_holiday', 'GSE134841'], #PC9 D19 DMSO
['GSM3972673', 19, 'PC9', 'Erl_Erl', 'Drug_holiday', 'GSE134841'], #PC9 D19 ERL
# GSE134842 (Beads on mix)
# ['GSM3972674', 0, 'PC9+U937', 'control', 'bead_mix', 'GSE134842'], #PC9 U937 mix
# ['GSM3972675', 0, 'PC9+U937', 'EpCAM+', 'bead_mix', 'GSE134842'], #EpCAM-positive by beads
# ['GSM3972676', 0, 'PC9+U937', 'CD45-', 'bead_mix', 'GSE134842'], #CD45-negative by beads
# GSE148466 (primary tissues)
# ['GSM4472055', 0, 'P8126999_primary', 'control', 'primary', 'GSE148466'], #P8126999
# ['GSM4472056', 0, 'P8127005_primary', 'control', 'primary', 'GSE148466'], #P8127005
# ['GSM4472057', 0, 'P8127011_primary', 'control', 'primary', 'GSE148466'], #P8127011
# GSE149214 (PC9 time course Mono-drug)
# ['GSM4494347', 0, 'PC9_rep1', 'control', 'multirep_PC9_mono', 'GSE149214'], #Day00b scRNA-seq
# ['GSM4494348', 11, 'PC9_rep1', 'Erl', 'multirep_PC9_mono', 'GSE149214'], #Day11b scRNA-seq
# ['GSM4494349', 11, 'PC9_rep2', 'Erl', 'multirep_PC9_mono', 'GSE149214'], #Day11c scRNA-seq
# GSE149215 (PC9 time course Multi-drug)
# ['GSM4494350', 3, 'PC9_rep1', 'Erl', 'multirep_PC9_multi', 'GSE149215'], #PC9D3-ERL1 scRNA-seq
# ['GSM4494351', 3, 'PC9_rep2', 'Erl', 'multirep_PC9_multi', 'GSE149215'], #PC9D3-ERL2 scRNA-seq
# ['GSM4494352', 3, 'PC9_rep1', 'Erl+Cri', 'multirep_PC9_multi', 'GSE149215'], #PC9D3-CRI-ERL1 scRNA-seq
# ['GSM4494353', 3, 'PC9_rep2', 'Erl+Cri', 'multirep_PC9_multi', 'GSE149215'], #PC9D3-CRI-ERL2 scRNA-seq
# ['GSM4494354', 3, 'PC9', 'Eto', 'multirep_PC9_multi', 'GSE149215'], #PC9Day3-Eto scRNA-seq
# GSE160244 (xenografts mice)
['GSM4869650', 3, 'PC9_xeno', 'control', 'Xenograft', 'GSE160244'], #Day 3 control
['GSM4869651', 3, 'PC9_xeno', 'Cri', 'Xenograft', 'GSE160244'], #Day 3 crizotinib
['GSM4869652', 3, 'PC9_xeno', 'Osi', 'Xenograft', 'GSE160244'], #Day 3 osimertinib
['GSM4869653', 3, 'PC9_xeno', 'Osi+Cri', 'Xenograft', 'GSE160244'] #Day 3 osimertinib+crizotinib
]
tab = pd.DataFrame(X, columns=['GEO', 'time', 'cell_line', 'perturbation', 'batch', 'subseries'])
tab.time = tab.time *24  # make hours
tab['replicate'] = [x.split('_')[-1] if 'rep' in x else None for x in tab.cell_line]
tab['cell_line'] = [x.split('_')[0]+'_xenograft' if 'xeno' in x else x.split('_')[0] for x in tab.cell_line]
tab = tab.set_index('GEO')

In [114]:
tab

,time,cell_line,perturbation,batch,subseries,replicate
GEO,,,,,,
GSM3972655,0,M14,control,M14D0vsD3Vem,GSE134838,None
GSM3972656,72,M14,Vem,M14D0vsD3Vem,GSE134838,None
GSM3972657,0,PC9,Erl,D0_D1_D2_D4_D9_D11,GSE134839,None
GSM3972658,24,PC9,Erl,D0_D1_D2_D4_D9_D11,GSE134839,None
GSM3972659,48,PC9,Erl,D0_D1_D2_D4_D9_D11,GSE134839,None
GSM3972660,96,PC9,Erl,D0_D1_D2_D4_D9_D11,GSE134839,None
GSM3972661,216,PC9,Erl,D0_D1_D2_D4_D9_D11,GSE134839,None
GSM3972662,264,PC9,Erl,D0_D1_D2_D4_D9_D11,GSE134839,None
GSM3972669,0,PC9,Erl,Drug_holiday,GSE134841,None


In [115]:
# subseries GSE149215 and GSE149214 not found
tab['filename'] = ''
for GEO in tab.index:
    files = [f.name for f in (TEMPDIR / 'AissaBenevolenskaya2021/').glob(f'{GEO}*.dge.txt')]
    assert len(files) == 1
    tab.loc[GEO, 'filename'] = files[0].replace('.dge.txt', '')

In [121]:
from scipy.sparse import csr_matrix

In [129]:
adatas = {}
for index, row in tab.iterrows():
    # load
    print(index)
    Y = pd.read_csv(TEMPDIR / f'AissaBenevolenskaya2021/{row.filename}.dge.txt', sep='\t', index_col=0)
    adata = sc.AnnData(csr_matrix(Y.values.T, dtype=int))
    adata.var_names = Y.index
    adata.obs_names = Y.columns
    adata.var_names_make_unique()

    # annotate by table
    for col in tab.columns:
        adata.obs[col] = row[col]
    adatas[index] = adata

GSM3972655
GSM3972656
GSM3972657
GSM3972658
GSM3972659
GSM3972660
GSM3972661
GSM3972662
GSM3972669
GSM3972670
GSM3972671
GSM3972672
GSM3972673
GSM4869650


KeyboardInterrupt: 

In [130]:
adata = sc.concat(adatas, index_unique='-')
# reform var, obs index names
adata.var.index = adata.var.index.rename('gene_symbol')
adata.obs.index = adata.obs.index.rename('cell_barcode')

# abbreviations to full compound name
for x,y in zip(['Vem', 'Erl', 'Cri', 'Eto', 'Osi'], ['vemurafenib', 'erlotinib', 'crizotinib', 'etoposide', 'osimertinib']):
    adata.obs.perturbation = [per.replace(x, y) for per in adata.obs.perturbation]

# further metadata
adata.obs['tissue_type']= ['cell_line' if x!='Xenograft' else 'cell_line' for x in adata.obs.batch]
adata.obs['cancer'] = True
adata.obs['perturbation_type'] = 'drug'
adata.obs['disease'] = ['melanoma' if x=='M14' else 'lung adenocarcinoma' if x=='PC9' else None for x in adata.obs.cell_line]
adata.obs['celltype'] = ['melanocytes' if x=='M14' else 'epithelial cells' if x=='PC9' else None for x in adata.obs.cell_line]
adata.obs['organism'] = 'human'

In [131]:
adata.obs.perturbation = adata.obs.perturbation.astype(str)
adata.obs['perturbation'] = [x.replace('+', '_') for x in adata.obs.perturbation]
adata.obs['nperts'] = [1+p.count('_')-p.count('control')-p.count('None') if type(p)==str else 0 for p in adata.obs.perturbation]

In [138]:
adata.obs.replicate=adata.obs.replicate.astype(str)

In [139]:
adata.write('/scratch/peidli/scPerturb/tmp.h5ad', compression='gzip')

In [136]:
pd.isna(adata.obs).sum(0)

time                     0
cell_line                0
perturbation             0
batch                    0
subseries                0
replicate            47518
filename                 0
tissue_type              0
cancer                   0
perturbation_type        0
disease                  0
celltype                 0
organism                 0
nperts                   0
dtype: int64

cell_barcode
AGATGTCGAAGC-GSM3972655    None
TACCTTTAGACT-GSM3972655    None
TGGCATTAAAGA-GSM3972655    None
CAAAGCGGAAAC-GSM3972655    None
CGACTATTATTA-GSM3972655    None
                           ... 
TTAATGGCTTCG-GSM3972673    None
TTACCGAAAAGC-GSM3972673    None
TTAGATTTTCTG-GSM3972673    None
TTAGCCCCCCGG-GSM3972673    None
TTCCACCGAAGC-GSM3972673    None
Name: replicate, Length: 47518, dtype: object

# Check

In [140]:
bdata = sc.read_h5ad('/scratch/peidli/scPerturb/AissaBenevolenskaya2021.h5ad')

In [141]:
bdata

AnnData object with n_obs × n_vars = 166589 × 11741
    obs: 'time', 'cell_line', 'perturbation', 'batch', 'subseries', 'replicate', 'filename', 'tissue_type', 'cancer', 'perturbation_type', 'disease', 'celltype', 'organism', 'nperts', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo'
    var: 'ncounts', 'ncells'

In [142]:
bdata.obs

,time,cell_line,perturbation,batch,subseries,replicate,filename,tissue_type,cancer,perturbation_type,disease,celltype,organism,nperts,ncounts,ngenes,percent_mito,percent_ribo
cell_barcode,,,,,,,,,,,,,,,,,,
AGATGTCGAAGC-GSM3972655,0,M14,control,M14D0vsD3Vem,GSE134838,None,GSM3972655_M14Day0,cell_line,True,drug,melanoma,melanocytes,human,0,17090,4155,3.200702,19.578701
TACCTTTAGACT-GSM3972655,0,M14,control,M14D0vsD3Vem,GSE134838,None,GSM3972655_M14Day0,cell_line,True,drug,melanoma,melanocytes,human,0,16504,4245,4.198982,13.784537
TGGCATTAAAGA-GSM3972655,0,M14,control,M14D0vsD3Vem,GSE134838,None,GSM3972655_M14Day0,cell_line,True,drug,melanoma,melanocytes,human,0,12414,3495,2.859675,15.998067
CAAAGCGGAAAC-GSM3972655,0,M14,control,M14D0vsD3Vem,GSE134838,None,GSM3972655_M14Day0,cell_line,True,drug,melanoma,melanocytes,human,0,12114,3697,4.862143,15.865940
CGACTATTATTA-GSM3972655,0,M14,control,M14D0vsD3Vem,GSE134838,None,GSM3972655_M14Day0,cell_line,True,drug,melanoma,melanocytes,human,0,12293,3665,4.303262,17.229317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTTTTTAGCAA-GSM4869653,72,PC9_xenograft,osimertinib_crizotinib,Xenograft,GSE160244,None,GSM4869653_xOsiCriz,cell_line,True,drug,NaN,NaN,human,2,125,97,0.800000,13.600000
TTTTTTTGCAAA-GSM4869653,72,PC9_xenograft,osimertinib_crizotinib,Xenograft,GSE160244,None,GSM4869653_xOsiCriz,cell_line,True,drug,NaN,NaN,human,2,179,135,2.234637,15.083799
TTTTTTTGTTGC-GSM4869653,72,PC9_xenograft,osimertinib_crizotinib,Xenograft,GSE160244,None,GSM4869653_xOsiCriz,cell_line,True,drug,NaN,NaN,human,2,33,17,57.575758,9.090909


In [144]:
bdata.obs.perturbation.unique()

['control', 'vemurafenib', 'erlotinib', 'erlotinib_DMSO', 'erlotinib_erlotinib', 'crizotinib', 'osimertinib', 'osimertinib_crizotinib']
Categories (8, object): ['control', 'crizotinib', 'erlotinib', 'erlotinib_DMSO', 'erlotinib_erlotinib', 'osimertinib', 'osimertinib_crizotinib', 'vemurafenib']